In [0]:
# Storage account details
storage_account_name = "heartdisease"
storage_account_key = "j9hfEzQWp/ibQ5HDycbM5MbUYdDU8PMy6i5IeaL//MVhOMJsRt84NyjMzFbiU4VVgv5N3kIQ+X+m+AStiUcfiw=="
container_name = "prediction"

# Configure Databricks to access Azure Blob Storage
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    storage_account_key
)

# Define the file path for your dataset in the Blob Storage container
file_path = "wasbs://prediction@heartdisease.blob.core.windows.net/heart_statlog_cleveland_hungary_final.csv"

# Read the dataset from Blob Storage
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows of the dataset
df.show(5)

# Print the schema of the dataset
df.printSchema()


+---+---+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
|age|sex|chest pain type|resting bp s|cholesterol|fasting blood sugar|resting ecg|max heart rate|exercise angina|oldpeak|ST slope|target|
+---+---+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
| 40|  1|              2|         140|        289|                  0|          0|           172|              0|    0.0|       1|     0|
| 49|  0|              3|         160|        180|                  0|          0|           156|              0|    1.0|       2|     1|
| 37|  1|              2|         130|        283|                  0|          1|            98|              0|    0.0|       1|     0|
| 48|  0|              4|         138|        214|                  0|          0|           108|              1|    1.5|       2|     1|
| 54|  1|              3|         

In [0]:
# Display basic statistics for numerical columns
df.describe().show()

# Count rows and columns
print(f"Total Rows: {df.count()}, Total Columns: {len(df.columns)}")


+-------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+
|summary|              age|                sex|   chest pain type|      resting bp s|       cholesterol|fasting blood sugar|       resting ecg|    max heart rate|    exercise angina|           oldpeak|          ST slope|             target|
+-------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+
|  count|             1190|               1190|              1190|              1190|              1190|               1190|              1190|              1190|               1190|              1190|              1190|               1190|
|   mean|53.72016806722689| 0.763865

In [0]:
# Count of target classes
df.groupBy("target").count().show()

from pyspark.sql.functions import corr

# Correlation between age and target
df.select(corr("age", "target")).show()

# Correlation between cholesterol and target
df.select(corr("cholesterol", "target")).show()


+------+-----+
|target|count|
+------+-----+
|     1|  629|
|     0|  561|
+------+-----+

+------------------+
| corr(age, target)|
+------------------+
|0.2620287733426452|
+------------------+

+-------------------------+
|corr(cholesterol, target)|
+-------------------------+
|      -0.1983663196793619|
+-------------------------+



In [0]:
# Count nulls in each column
df.select([df[c].isNull().alias(c) for c in df.columns]).show()

df = df.fillna({"cholesterol": 0, "resting bp s": 120})



+-----+-----+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
|  age|  sex|chest pain type|resting bp s|cholesterol|fasting blood sugar|resting ecg|max heart rate|exercise angina|oldpeak|ST slope|target|
+-----+-----+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
|false|false|          false|       false|      false|              false|      false|         false|          false|  false|   false| false|
|false|false|          false|       false|      false|              false|      false|         false|          false|  false|   false| false|
|false|false|          false|       false|      false|              false|      false|         false|          false|  false|   false| false|
|false|false|          false|       false|      false|              false|      false|         false|          false|  false|   false| false|
|false

In [0]:
from pyspark.sql.functions import when

# Convert binary 'sex' column
df = df.withColumn("sex", when(df["sex"] == 1, 1).otherwise(0))

# One-hot encode 'chest pain type'
df = df.withColumn("chest_pain_1", when(df["chest pain type"] == 1, 1).otherwise(0))
df = df.withColumn("chest_pain_2", when(df["chest pain type"] == 2, 1).otherwise(0))


In [0]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

# Combine columns for scaling
assembler = VectorAssembler(inputCols=["age", "cholesterol", "max heart rate"], outputCol="features")
df = assembler.transform(df)

# Apply MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
df = scaler.fit(df).transform(df)


In [0]:
from pyspark.ml.classification import LogisticRegression

# Select scaled features and target for training
ml_data = df.select("scaled_features", "target")

# Split the data into training and testing sets
train_data, test_data = ml_data.randomSplit([0.8, 0.2], seed=123)

# Train a logistic regression model
lr = LogisticRegression(featuresCol="scaled_features", labelCol="target")
lr_model = lr.fit(train_data)

# Evaluate the model
test_results = lr_model.evaluate(test_data)
accuracy = lr_model.summary.accuracy
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 0.6957894736842105


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Get predictions from the model
predictions = lr_model.transform(test_data)

# Evaluate accuracy
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy = accuracy_evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Evaluate precision (for multiclass)
precision_evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="weightedPrecision")
precision = precision_evaluator.evaluate(predictions)
print(f"Weighted Precision: {precision}")

# Evaluate recall (for multiclass)
recall_evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="weightedRecall")
recall = recall_evaluator.evaluate(predictions)
print(f"Weighted Recall: {recall}")

# Evaluate F1-score (for multiclass)
f1_evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(predictions)
print(f"F1 Score: {f1}")

# Evaluate ROC-AUC (for binary classification)
roc_auc_evaluator = BinaryClassificationEvaluator(labelCol="target", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = roc_auc_evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")


Accuracy: 0.675
Weighted Precision: 0.6806616194865043
Weighted Recall: 0.675
F1 Score: 0.6708730158730158
ROC-AUC: 0.761865054547981


In [0]:
wasbs://prediction@heartdisease.blob.core.windows.net/processed/predictions.csv


  File <command-3559252805844565>, line 1
    wasbs://prediction@heartdisease.blob.core.windows.net/processed/predictions.csv
          ^
SyntaxError: invalid syntax
